In [1]:
import pandas as pd
import json
import random
import os
from pathlib import Path

In [2]:
CRO_train = [
    {'text': "La présentation clinique et histologique sont concordants avec une atteinte de psoriasis. L'évolution de cette maladie peut varier selon le stade et la localisation des lésions. Il est important de mettre en place une soins appropriés pour améliorer la qualité de vie du patient et ralentir la progression de la maladie.", 'diag': "psoriasis"},
    {'text': "L'analyse histologique a révélé une prolifération cellulaire maligne en épithélioma spiculeux, typique de mélanome. Les cellules tumorales étaient polymorphes, avec nucléus fusiformes et hyperchromatiques, et cytoplasme variably basophilique. Des mitoses atypiques étaient présentes. Les cellules s'infiltraient profondément dans les couches dermiques, envahissant le muscle sousjacent localement. Une importante infiltration inflammatoire comprenait principalement des lymphocytes T activés et des neutrophiles. L'immunomarquage S-100 positif confirmé la nature melanocytique des cellules tumorales.", 'diag': "mélanome"},
    {'text': "La biopsie histologique révèle une rosacée moderée sans atypie ni signes de malignité. L'évolution clinique devra être surveillée et traitement topique approprié instauré selon la gravité de la symptomatologie.", 'diag': "rosacée"},
    {'text': "Le tissu examiné correspond à un cas de lupus érythémateux, établi sur base de la présence de nécrose folliculaire, lymphocytose interstitielle prédominante, déposition d'immuns déposés et absence de cornées granulosae. Cette affection auto-immune peut entraîner des complications cutanées et systémiques, il est important de mettre en place une surveillance régulière et de conseiller une thérapie appropriée afin de contrôler la progression de la maladie.", 'diag': "lupus érythémateux"},
    {'text': "À l'examen microscopique, on note la présence d'un infiltrat lymphocytaire superficiel et profond dans les couches papillaires et reticulaires de la peau. Les kératinocytes supra-basaux montrent une hyperkératose et des acanthoses bipolares. Des bandes subepidermiques de liquide mucosal sont également visibles. Les fibroblasts montrent des modifications atypiques avec augmentation de leur taille et hyperchromatisme nucléaire. Il y a aussi des dépôts amorphes de matériel éosinophile localisés entre les collagène I et III. Ces changements correspondent aux aspects histologiques typiques d'un lichen plan.", 'diag': "lichen plan"},
    {'text': "Dans son ensemble, ces aspects histologiques sont compatibles avec un carcinome basocellulaire invasif. Le stade TNM correspondrait probablement à un T1a N0 BCLC A selon les critères de l'American Joint Committee on Cancer (AJCC). Une excision chirurgicale complète ou une radiothérapie peuvent être proposées comme traitements possibles.", 'diag': "carcinome basocellulaire"},
    {'text': "Cette exérèse histologique confirme le diagnostic de lichen scléreux localisé au niveau du dos du patient. Il est important de noter que cette maladie peut évoluer vers des formes plus agressives si aucun traitement n'est mis en place. Nous recommandons aux soins médicaux concernés de mettre en place un suivi régulier afin d'éviter toute progression locale ou systémique potentiellement grave. Si nécessaire, une consultation spécialiste en dermatologie sera proposée pour discuter des options thérapeutiques appropriées.", 'diag': "lichen scléreux"},
    {'text': "Le diagnostic histologique de cette pièce correspond à une pityriasis rosé de Gibert. Cette condition est caractérisée par une hyperprolifération des kératinocytes dans les zones de transition entre l'épiderme et le derme, entraînant une hyperkératose acanthotique et une inflammation subjacente. Ce processus est généralement considéré comme étant benigne et auto-limitée. Il n'y avait pas de signe de malignité ou d'autres anomalies pathologiques observées dans ce prélèvement.", 'diag': "pityriasis rosé de Gibert"},
    {'text': "La lésion histologique correspond à une Maladie de Duhring-Brocq. L'absence de signes cliniques associés à cette affection systémique permet de considérer ce cas comme isolé cutané. Il est important de notifier que cette maladie peut évoluer vers des complications plus graves si elle n'est pas traitée appropriément. Nous recommandons une surveillance régulière afin de dépister toute évolution potentielle.", 'diag': "maladie de Duhring-Brocq"},
    {'text': "Dans cette observation, nous constatons un sarcome à cellules fusiformes de Kaposi, confirmé par l'expression immunohistochimique positive pour le CD30 et la protéine K (KP). Ce diagnostic est communément associé au VIH et à l'immunodéficience. Il est important de notifier aux soins médicaux concernés ce diagnostic afin de mettre en place une surveillance étroite et de préparer une éventuelle thérapie antirétrovirale si nécessaire. Nous recommandons également une recherche systématique de lésions similaires dans d'autres sites cutanés et viscéraux.", 'diag': "sarcome à cellules fusiformes de Kaposi"},
    {'text': "L'examen histologique a révélé une infiltration tumorale profonde du derrière du derme et de l'hypoderme par des cellules fusiformes atypiques, arrondies ou polygonalement disposées, monstrant des nucléoles vifs et irreguliers. Des mitoses actives étaient présentes. Les cellules fusiformes formaient des fascicules myxoïdes et des cordons intravasculaires. Des karyorrhées ainsi que des éosinophiles intracytoplasmiques étaient observés. L'immunomarquage révéla une expression positive pour le CD30 et la protéine K (KP) tandis qu'il était négatif pour les marqueurs lymphocytaires tels que le CD20, le CD3 et le CD45.", 'diag': ""},
    {'text': "L'examen histologique révèle une prolifération glandulaire bien différenciée formant des structures tubulaires et aciniquements disposées en nidus dispersés dans un stroma fibro-adipose. Des cellules épithéliales cuboïdales polymorphes sont observées aux bords des glands et dans les interstices entre eux. Ces dernières présentent des nucléoles centrés ou subcentrés et peuvent montrer des vacuolations cytoplasmiques. Les glands contiennent des cellules mucosées apicales et des cellules goblet. Aucun signe inflammatoire notable n'est détectable. Ce tableau correspond à celui d'un hamartome intestinal colique.", 'diag': "hamartome"},
    {'text': "Sur le plan histologique, l'échantillon présente une invasion submucosa profonde (T3) d'un adénocarcinome du côlon. Ce dernier est constitué de glandules malformées formant des structures tubulo-glandulaires discontinues, avec des cellules épithéliales cuboïdes à columnaires arrondies, atypiques, et hyperchromatiques. Des cellules mucines produisantes sont observées dans les glandules et entre elles. Des mitoses sont visibles dans les différentes couches de l'épaisseur tumorale. Des signes d'invasion peripériqueurs sont présents, ainsi qu'une importante infiltration inflammatoire comprenant principalement des lymphocytes plasmatiques. Aucun signe d'angio-invasion ou de lymphovascularisation n'est observable.", 'diag': "adénocarcinome"},
    {'text': "La biopsie histologique du colon sigmoïdien de Mme Susanne Renault a révélé une diverticulite avec un épithélium épithéloïde normal et des diverticules coliques assez communs dans cet emplacement. Aucune anomalie pathologique significative n'a été détectée. Ce résultat correspond à une diverticulose colique asymptomatique.", 'diag': "diverticulite"},
    {'text': "Le prélèvement histologique indique une colite ischémique avec necrosis intestinale et changements dysplasiques précancéreux. Cette affection est liée à une insuffisance rétrograde du flux sanguin vers le côlon, qui peut résulter d'une compression vasculaire ou d'une hypovolémie. En raison de la gravité des modifications histologiques observées, il est fortement recommandé de mettre en place une surveillance régulière afin d'éliminer tout risque de développement ultérieur de cancer colorectal.", 'diag': "colite ischémique"}, 
    {'text': "Histologiquement, la section montre une infiltration tumorale profonde du muscle lisse du mur intestinal par des cellules arrondies à fusiformes, mononucleées, avec cytoplasmique granulé caractéristique (Figure 1). Ces cellules présentaient un nucléole central et des mitoses modérément fréquentes. Des nests de cellules tumorales s'étendaient jusqu'au bord de la section, confirmant l'absence de résidus tumoraux. Il y avait également des ganglions lymphatiques locaux invasiés par les cellules tumorales (Figure 2). Immunohistochimiquement, les cellules tumorales exprimaient la chromogranine A et la synaptophysine, confirmant le diagnostic de tumeur carcinoïde du côlon (Figure 3).", 'diag':"tumeur carcinoïde"},
    {'text': "Dans sa globalité, cette étude histopathologique confirme le diagnostic initial de tumeur neuroendocrine. Le CNE atteignait tous les plans épithélio-muscleux et avait envahi profondément les structures adjacentes. L'absence de résidus tumoraux à proximité des bords du spécimen permettrait de considérer que les marges chirurgicales sont suffisamment larges. Toutefois, il convient de réaliser une évaluation immunohistochimique complète afin de déterminer si ce CNE est fonctionnel ou non et de définir son stade évolutif.", 'diag': "tumeur neuroendocrine"}, 
    {'text': "Dans cette biopsie de colon, des foyers de dysplasie tubulovillosaire modérée à grave sont observés, qui correspondent aux critères histologiques du syndrome de Lynch. Cette anomalie précoce indique un risque accru de développement ultérieur d'un carcinome colorectal. Une surveillance étroite et régulière est recommandée pour ce patient afin d'identifier toute évolution vers un cancer colorectal plus tôt plutôt que tarder.", 'diag': "syndrome de Lynch"}, 
    {'text': "L'échantillon étudié présente une inflammation chronique des parois intestinales du côlon avec des signes typiques de la maladie de Crohn. Les sections transversales montrent une épithélium colique normal ou légèrement atrophié associé à des infiltrats lympho-plasmacellulaires sous-mucosaux et dans les plexus mucosaux. Des granulomes sont observés dans certaines régions, composés principalement de lymphocytes T et macrophages multinucléés. Des ulcères profonds traversent tout le thickness muréal du côlon, présentant des bords irreguliers et des cryptites déformées. Des fibrosis focale est présente dans certains endroits. Il y a également des signes d'inflammation vasculaire avec thromboses veineuses et artérielles intramurales. Le muscle lisse est intact.", 'diag': "maladie de Crohn"},
    {'text': "Dans sa totalité, cet examen histopathologique confirme le diagnostic préliminaire de cancer colorectal héréditaire non polyposique basé sur l'âge jeune du patient et son antécédent familial. Ce type particulier de carcinome colorectal est associé à des mutations génétiques spécifiques et peut causer des cancers multiples et récurrents. En raison de ce diagnostic, il est fortement recommandé aux membres de la famille immédiate de M. Joseph Berthelot de faire des tests ADN afin de déterminer si ils portent également la mutation responsable. Une proton pump inhibitor (PPI) devrait être prescrit régulièrement pendant la période post-biopsie pour réduire le risque de complications ulcéreuses liées à l'utilisation de la biopsie forceps.", 'diag': "cancer colorectal héréditaire non polyposique"},
    {'text': "Lors de cette intervention chirurgicale, nous avons procédé à une colectomie totale avec anastomose colo-rectale élevée. Un polype de grande taille a été identifié dans le côlon transversal, mesurant environ 4 cm de diamètre. Ce polype était localisé à environ 50 cm proximal au sigmoïde. Les marges histologiques étaient claires. Aucune autre masse tumorale ou anomalie macroscopiquement significative n'a été observée.", 'diag': ""}]

CRO_valid = [
    {'text': "Cet examen histopathologique confirme le diagnostic clinique de psoriasis cutané. Il existe une importante activation de l'immunité cellulaire et inflammatoire, notamment une forte infiltration de lymphocytes T et B, neutrophiles et polynucléaires neutrophiles, associée à une hyperprolifération kératinocytaire. Cette activité inflammatoire expliquerait les symptômes cliniqques observés chez le patient.", 'diag': "psoriasis"},
    {'text': "Dans sa totalité, ces aspects sont compatibles avec un mélanome cutané de stade I-II, sans évidence de métastases loco-régionales ou distantes. Il est important de communiquer ce résultat aux soignants impliqués dans le traitement de M. Alexandre Clément afin qu'ils puissent prendre des décisions appropriées en fonction de ses caractéristiques clíniques et histologiques.", 'diag': "mélanome"},
    {'text': "La biopsie histologique révèle une rosacée modérée avec une activation inflammatoire locale. La présence de vasculite focale suggère une forme plus aggravée de cette maladie. Une approche thérapeutique adaptée devrait être considérée afin de contrôler les symptômes et prévenir toute progression potentielle vers une forme plus sévère.", 'diag': "rosacée"},
    {'text': "L'examen histologique révèle une infiltrat granulomateux profond dans les couches dermiques et sous-cutanées. Des cellules géantes multinucléées sont observées entourées de mononucléaires et de lymphocytes T. Des dépôts amorphes de matériel éosinophile sont également visibles. Les karyorrhées et les débris nucléaires indiquent une activité inflammatoire intense. Des fibroses scléreuses sont observées entre les structures granulomateuses. Des dépôts immuns complément fixés (ICF) sont détectés autour des vaisseaux sanguins et des glandes sudoripares. Une biopsie concomitante a confirmé le diagnostic de lupus érythémateux.", 'diag': "lupus érythémateux"},
    {'text': "Dans son ensemble, ces observations histologiques sont compatibles avec un diagnostic de lichen plan cutané. Ce processus inflammatoire autoimmunitaire peut entraîner divers symptômes cliniques, allant de petites plaques prurigineuses à des ulcères profonds. Une confirmation diagnostique complémentaire devrait être envisagée si nécessaire (test immunohistochimique ou étude immunofluorescente). Un traitement adapté devrait être proposé au patient en fonction de l'étendue et de la gravité de sa maladie.",'diag': "lichen plan" },
    {'text': "À l'examen microscopique, on retrouve une prolifération épidermoïde maligne caractéristique d'un carcinome basocellulaire. Les cellules tumorales ont un aspect cuboïde à polygonal, avec nucléoles irréguliers et hyperchromatiques. Des mitoses sont visibles dans les couches inférieures de l'épaisseur de l'épiderme. Il y a présence de dégranulation kératinocytaires et formation de grenades mucosées. Les follicules sont intacts, mais il existe des infiltrats inflammatoires perivascular autour des vaisseaux sanguins.", 'diag': "carcinome basocellulaire"},
    {'text': "Dans son ensemble, ces résultats sont compatibles avec une excision complète d'un lichen scléreux. Une surveillance clinique régulière est recommandée afin de détecter toute possible récidive locale ou développement de nouveaux foyers.", 'diag': "lichen scléreux"}, 
    {'text': "Le tableau clinique et les résultats histologiques correspondent à une pityriasis rosé de Gibert. Cette entité est caractérisée par une éruption papulo-plaquiculaire, généralement bilatérale, qui peut toucher principalement la région toraco-abdominale. L'étiologie exacte reste inconnue, mais il est admis que cette affection est liée à une hypersensibilité à certains agents exogènes comme certaines plantes, des médicaments ou des produits chimiques. Il est important de rechercher une cause possible et de conseiller au patient d'éviter toute exposition à ces agressions potentielles.", 'diag': "pityriasis rosé de Gibert"},
    {'text': "L'examen histologique montre une prolifération tumorale formée majoritairement par des cellules sphériques ou ovalaires, mitotiques, avec des nucléoles intermédiaires et des cytoplasmes riches en glycogène (figure 1). Des fibroblasts normaux sont observés entre les masses cellulaires. Il y a présence de vaisseaux sanguins et lymphatiques intacts. Les bords de l'excision présentent une zone de transition vers la kératinocytose normale (figure 2). L'immunomarquage HMB45 est positif, confirmant le diagnostic de maladie de Duhring-Brocq (figures 3 et 4).", 'diag': "maladie de Duhring-Brocq"},
    {'text': "L'analyse histologique a révélé une prolifération tumorale dense et hétérogène formée majoritairement par des cellules fusiformes et spindloides, disposées en fascicules irréguliers et intercalées entre des fibroblastes normaux (figure 1). Des cellules multinucleaires géantes étaient également présentes dans les zones centrale et périphérique de la tumeur. Les mitoses étaient abondantes et atypiques. Des lymphocytes inflammatoires were observés entre les fascicules tumoraux et autour des vaisseaux sanguins. Immunohistochimie révéla que ces cellules étaient positives pour le marqueur CD30 et negative pour le marqueur S-100 protein, ce qui est conséquent avec un diagnostic de sarcome à cellules fusiformes de Kaposi (Figure 2)", 'diag': "sarcome à cellules fusiformes de Kaposi"}, 
    {'text': "Sur section transversale, l'épaisseur totale de l'échantillon est estimée à environ 6 mm. L'ulcération est localisée dans la couche épidermique et profondement infiltrative jusqu'à la membrane basale. Des glandes sebacées dilatées et hyperplasiques sont observées dans tout le derme. Il y a également une importante infiltration inflammatoire, principalement composée de lymphocytes T CD4+ et CD8+, ainsi que d'autres cellules inflammatoires telles que des neutrophiles et des plaques immunes. Des vascularites fibrinoïdes sont observées autour des glandes sebacées dilatées. Des mitoses sont présentes dans les kératinocytes suprabasaux.", 'diag': ""},
    {'text': "Le prélevé histologique indique un hamartome colonique bien circonscrit sans signes d'invasion ni de métastases. Cette anomalie est généralement considérée comme bénigne, mais elle peut occasionner des complications cliniques telles que l'obstruction intestinale ou la saignement. Une surveillance régulières est recommandée pour détecter toute évolution potentielle vers une formation plus grande ou maligne.", 'diag': "hamartome"},
    {'text': "Histologiquement, l'échantillon montrait un adénocarcinome infiltrant les couches musculaires propres et submucosales. Des ganglions régionaux étaient impliqués par métastases carcinomateuses. Les cellules tumorales présentaient des nucléoles hyperchromatiques et des cytoplasmes pauvres. Il y avait également des zones d'acide mucoglycanique intracytoplasmique. Des atypies mitotiques et des figures binucleées étaient observées. Les voies veineuses et lymphatiques étaient empaquetées par des cellules tumorales.", 'diag': "adénocarcinome"},
    {'text': "Cette biopsie histologique confirme le diagnostic de diverticulite du côlon avec complications comprenant abscès péri-diverticulaire et thrombose veineuse. La chirurgie réalisée était appropriée pour traiter cette condition compliquée.", 'diag': "diverticulite"},
    {"text": "La biopsie histologique du spécimen de résection indique une colite ischémique sans signe de cancer.", 'diag': "colite ischémique"},
    {'text': "À l'examen microscopique, il est possible d'observer une prolifération tumorale formée majoritairement par des cellules arrondies à polygonalaires, mononucléées, avec cytoplasmique granulomateuse et nucléus rondelets ou ovales. Les mitoses sont relativement abondantes (environ 8/mm²). Des groupements de cellules nettoyées sans noyaux sont également présents. Les espaces intercellulaires contiennent des mucines acides qui réagissent positivement aux stains Alcian Blue et Periodic Acid Schiff (PAS). Il y a aussi des infiltrats inflammatoires lymphoïdes modéréments abondants dans les espaces interstitielles. Immunohistochimiquement, ces cellules expriment chromogranine A et synaptophysine, confirmant ainsi le diagnostic de tumeur carcinoïde du côlon.", 'diag': "tumeur carcinoïde"},
    {'text': "Les différentes analyses réalisées permettent de diagnostiquer une tumeur neuroendocrine bien différencié du côlon, type B2, a été identifié sur le spécimen chirurgical de colectomie chez M. Joseph Berthelot. Cette tumeur est caractérisée par sa croissance en nests et acini, son aspect histologique typique et ses marqueurs immunohistochimiques positifs pour chromogranine A et synaptophysine. L'extension locale est confirmée par l'invasion musculaire profonde et peripérique, ainsi que par l'invasion ganglionnaire locale. Une surveillance régulière sera nécessaire afin d'évaluer la possibilité d'extensions subliminale ou distante.", 'diag': ""},
    {'text': "Dans sa totalité, ces résultats histologiques sont concordants avec un diagnostic de syndrome de Lynch. Une confirmation moléculaire complémentaire sera effectuée afin de préciser le type particulier de ce syndrome.", 'diag': "syndrome de Lynch"},
    {'text': "L'étude histologique révéla une maladie de Crohn diffuse et active, touchant tous les segments analysés du côlon. Les sections transversales montraient une inflammation granulomateuse associée à des ulcérations profondes et des cryptites suppuratives. Des infiltrats inflammators riches en neutrophiles, eosinophiles et plasmocytes étaient présents autour des vaisseaux sanguins et des ganglions lymphatiques. Des filaments de collagène furent détectés dans les couches musculaires propices aux ulcérations. Des granulomes épithéloïdes multicellulaires étaient également visibles, contenant des géants cellulaires et des lymphocytes T CD4+ et CD8+. Des bacilles Gram positifs furent identifiés dans certains ulcères.", 'diag': "maladie de Crohn"},
    {'text': "Dans sa totalité, ces observations sont compatibles avec un cancer colorectal héréditaire non polyposique, stade T3N0Mx. Une confirmation moléculaire est recommandée afin de préciser le type spécifique de mutation responsable de cette affection. Un suivi régulier et étroit devra être instauré pour évaluer l'évolution de cette pathologie et éventuellement mettre en place des mesures thérapeutiques appropriées.", 'diag': "cancer colorectal héréditaire non polyposique"},
    {'text': "Le diagnostic final de ce cas est celui d'une colectomie pour polypose adénomateuse familiale (FAPC) du côlon, confirmé par la présence d'un grand tumorama invasif dans le côlon sigmoïdien inférieur ainsi que de nombreux petits polypes adhérents dans les segments adjacents. L'extension locale et l'agressivité histologique du tumeur justifieront une approche thérapeutique agressive, y compris une chirurgie extensiva et une surveillance régulière afin d'identifier toute récidive ou développement de nouveaux polypes.", 'diag': "polypose adénomateuse familiale"},
    {'text': "Le spécimen colique reçu mesurait environ 12 cm de longueur totale, comprenant un segment colique droit de 6 cm, un transverse de 3 cm et un sigmoïde gauche de 3 cm. La masse tumorale principale était localisée dans le côlon sigmoïdien inférieur, mesurant approximativement 5 cm x 4 cm x 3 cm. Sa surface était irrégulière et hétérogène, avec des zones ulcéreuses et des areas rouges. Des polypes adhérarent à la masse principale ont également été identifiés dans les segments adjacents du côlon transvers et droit.", 'diag':''}
]

CRO_test = [
    {'text': "Cet examen histologique confirme le diagnostic initial d'un hémangiome du foie. Ce type de tumeur est bénin et ne nécessite généralement pas de traitement spécifique si elle reste stable sur imaging. Une surveillance régulière est recommandée pour surveiller eventuellement une croissance ou une transformation maligne potentielle.", 'diag': "hémangiome"},
    {'text': "Le diagnostic histologique suggère sur base de ces observations est un cystadénome biphasique du foie. Ce type de tumoréalisation est rare mais bénin et ne nécessite généralement pas de traitement chirurgical si le diagnostic est confirmé. Cependant, il convient de poursuivre l'exploration clinique pour exclure toute autre cause de symptômes abdomino-péritonéaux potentiellement liée à cette observation.", 'diag': "cystadénome"},
    {'text': "Dans la mesure du possible, cette étude histologique fournit des informations supplémentaires sur le diagnostic initial de hépatoblastome basé sur les examens cliniques et radiologiques. L'aspect histologique correspondant à un hépatoblastome de type fetal indique que ce tumeur est généralement plus favorable aux traitements et qu'elle peut avoir une bonne réponse thérapeutique. Cependant, il est important de noter que cette analyse histologique ne remplace pas complètement les autres modalités de diagnostic et de staging, telles que l'imagerie par résonance magnétique (MRI), l'échographie abdominopelvienne et l'analyse moléculaire.", 'diag': "hépatoblastome"},
    {'text': "Les caractéristiques histologiques de cette biopsie sont compatibles avec un cholangiocarcinome intraductal infiltratif (ICC) ou un carcinome papillaire intra-hépatique (IPHCC). Cette maladie est généralement agressive et peut être difficile à traiter. Une confirmation par immunohistochimie et électron microscopie seront nécessaires pour établir un diagnostic définitif.", 'diag': "cholangiocarcinome"},
    {'text': "À l'examen microscopique, la tumeur est constituée d'une prolifération tumorale bien circonscrite formant des glandes tubulaires et acini. Les cellules épithéliales sont cuboides à columnaires, monstrueuses, avec cytoplasmes granuleux et nucléoles centrés. Des mitoses sont visibles dans les acines. Une forte infiltration inflammatoire lymphocytaire est présente autour des vaisseaux sanguins. Immunohistochimiquement, les cellules expriment chromogranine A et synaptophysine, confirmant ainsi le diagnostic de tumeur neuroendocrine du côlon. Il y a invasion musculaire profonde (pT3) et absence de métastases ganglionnaires (N0).", 'diag': "tumeur neuroendocrine"},
    {'text': "Le diagnostic histologique final est un carcinome squirrheux invasif du foie. Cette tumeur était bien circonscrite lors de l'intervention chirurgicale, mais des marges histologiques doivent être évaluées pour déterminer si une résiduelle tumorale existe. Une surveillance régulières est recommandée afin de détecter toute récidive potentielle.",'diag': "carcinome squirrheux"}
]

In [3]:
def find_occurrences(texte: str, mot: str) -> list:
    ''' 
    trouve toutes les occurrences d'un mot/phrase en renvoyant les index de début
    - entrée : - texte (str) => texte dans lequel le mot doit être recherché
               - mot (str) => mot recherché dans le texte
    - sortie : list de tous les index de début de localisation dans le texte
    '''
    occurrences = []
    index = -1
    while True:
        texte = texte.lower()
        # trouve toutes les occurences d'un mot dans le texte jusqu'à la fin
        index = texte.find(mot.lower(), index + 1)
        if index == -1:
            break

        index_fin = index + len(mot)

        # vérifie par quoi se termine le mot pour éviter les mots dérivés du mot recherché
        if texte[index_fin] in [' ', ',', '.', ')', ':']:
            occurrences.append(index)
        else:
            print("mot suivi d'une lettre", mot, '\n', texte[index:])

        if -1 in occurrences:
            raise
    return occurrences

def normalisation_texte(texte: str): 
    '''
    mise en minuscule et suppression des retours à la ligne \n.
    '''   
    texte_normalise = texte.replace('\n', ' ')

    return texte_normalise

In [4]:
def csv_to_json(df: list, file_json : Path) -> str:
    '''
    Génération d'un fichier JSON qui rassemble le texte, les index de début et de fin de chaque mot associé à leur label
    - entrées : - df (pd.DataFrame): dataframe ressemblant le texte et l'ensemble des mots ou groupes de mots recherchés dans le texte.
                - file_json (str): adresse de lecture et de d'enregistrement du fichier JSON
    - sortie : un fichier JSON
    '''
    df_to_json = []

    for i in range(0, len(df)):
        # variables qui comptent le nb de fois qu'une valeur est trouvé dans le texte
        compteur_diag = 0

        list_ner = []
        dict_ner = {'entities': []}
        # text = normalisation_texte(df.CRO[i])
        text = df[i]['text']

        # ajout du texte au dictionnaire
        list_ner.append(text)

        # recherche les occurrences des diagnostics
        diag = df[i]['diag'].lower()
        print('diagnostic recherché', diag)
        if diag != "":
            diag_index = find_occurrences(text, diag)

            for k in diag_index:
                end = k + len(diag) 
                compteur_diag += 1       
                dict_ner['entities'].append([k, end, 'DIAG'])
        else:
            pass
        
        # ajout les différents index de début et de fin au dictionnaire
        dict_ner['entities'] = sorted(dict_ner['entities'], key=lambda x: x[2])
        list_ner.append(dict_ner)              
        df_to_json.append(list_ner)

    # ouverture du fichier JSON
    to_json = json.dumps(df_to_json, ensure_ascii=False)

    with open(file_json, 'w', encoding="utf-8") as fichier:
        # écriture du fichier JSON, ajout si existe déjà !
        fichier.write(to_json)
    return len(df_to_json)

In [6]:
print('génération du fichier json train')
taille_json_train = csv_to_json(CRO_train, '../generate_model/assets/train.json')
print(f'La data train possède {len(CRO_train)}, le JSON train contient {taille_json_train} entrées.')

print('génération du fichier json valid')
taille_json_valid = csv_to_json(CRO_valid, '../generate_model/assets/dev.json')
print(f'La data valid possède {len(CRO_valid)}, le JSON valid contient {taille_json_valid} entrées.')

print('génération du fichier json test')
taille_json_test = csv_to_json(CRO_test, '../generate_model/assets/test.json')
print(f'La data test possède {len(CRO_test)}, le JSON test contient {taille_json_test} entrées.')

génération du fichier json train
diagnostic recherché psoriasis
diagnostic recherché mélanome
diagnostic recherché rosacée
diagnostic recherché lupus érythémateux
diagnostic recherché lichen plan
diagnostic recherché carcinome basocellulaire
diagnostic recherché lichen scléreux
diagnostic recherché pityriasis rosé de gibert
diagnostic recherché maladie de duhring-brocq
diagnostic recherché sarcome à cellules fusiformes de kaposi
diagnostic recherché 
diagnostic recherché hamartome
diagnostic recherché adénocarcinome
diagnostic recherché diverticulite
diagnostic recherché colite ischémique
diagnostic recherché tumeur carcinoïde
diagnostic recherché tumeur neuroendocrine
diagnostic recherché syndrome de lynch
diagnostic recherché maladie de crohn
diagnostic recherché cancer colorectal héréditaire non polyposique
diagnostic recherché 
La data train possède 21, le JSON train contient 21 entrées.
génération du fichier json valid
diagnostic recherché psoriasis
diagnostic recherché mélanome
d